In [1]:
# main_solver.py

import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict
import sys

class DataParser:
    """
    Responsabilidade: Ler e analisar o arquivo de instância de forma otimizada.

    Design: Esta classe pré-processa os dados para criar 'índices invertidos'
    (item_to_orders, item_to_aisles). Isso acelera drasticamente a fase de 
    construção do modelo, que era nosso gargalo anterior.
    """
    def __init__(self, filepath):
        # O caminho para o arquivo de instância a ser lido.
        self.filepath = filepath
        
        # Estrutura para guardar a qtde total de itens por pedido, útil para as restrições de tamanho.
        self.orders_total_units = {}
        
        # Limites inferior e superior para o tamanho da wave.
        self.lb = 0
        self.ub = 0
        
        # --- ESTRUTURAS OTIMIZADAS (ÍNDICES INVERTIDOS) ---
        # Usamos defaultdict para simplificar a adição de novos itens sem checagens extras.
        # Mapeia um item para uma lista de pedidos que o contêm: {item_id: [(order_id, qty), ...]}
        self.item_to_orders = defaultdict(list)
        # Mapeia um item para uma lista de corredores que o contêm: {item_id: [(aisle_id, qty), ...]}
        self.item_to_aisles = defaultdict(list)
        
        # Conjunto de todos os IDs de itens, pedidos e corredores para fácil iteração.
        self.all_items = set()
        self.all_orders = set()
        self.all_aisles = set()


    def parse(self):
        """
        Executa a leitura e o pré-processamento do arquivo de entrada.
        """
        print(f"INFO: Iniciando a análise otimizada do arquivo: {self.filepath}")
        try:
            with open(self.filepath, 'r') as f:
                # Lê a primeira linha com as contagens gerais.
                num_orders, _, num_aisles = map(int, f.readline().split())

                # --- Processamento dos Pedidos ---
                for i in range(num_orders):
                    self.all_orders.add(i)
                    line = list(map(int, f.readline().split()))
                    total_units = 0
                    # Itera sobre os pares de (item, quantidade) na linha.
                    for j in range(line[0]):
                        item_id = line[1 + 2*j]
                        quantity = line[2 + 2*j]
                        
                        # AQUI: Populamos o índice invertido.
                        self.item_to_orders[item_id].append((i, quantity))
                        
                        total_units += quantity
                        self.all_items.add(item_id)
                    self.orders_total_units[i] = total_units

                # --- Processamento dos Corredores ---
                for i in range(num_aisles):
                    self.all_aisles.add(i)
                    line = list(map(int, f.readline().split()))
                    # Itera sobre os pares de (item, quantidade) na linha.
                    for j in range(line[0]):
                        item_id = line[1 + 2*j]
                        quantity = line[2 + 2*j]
                        
                        # AQUI: Populamos o índice invertido dos corredores.
                        self.item_to_aisles[item_id].append((i, quantity))
                        self.all_items.add(item_id)

                # Lê a última linha com os limites da wave.
                self.lb, self.ub = map(int, f.readline().split())
            
            print("INFO: Análise do arquivo concluída com sucesso.")
            return True
        except FileNotFoundError:
            print(f"ERRO: Arquivo de instância não encontrado em '{self.filepath}'")
            return False
        except Exception as e:
            print(f"ERRO: Ocorreu um erro inesperado durante o parsing: {e}")
            return False


class WaveOptimizer:
    """
    Responsabilidade: Construir, resolver e extrair a solução do modelo de otimização.
    """
    def __init__(self, data, lambda_penalty=1.0, stagnation_node_limit=5000):
        # O objeto 'data' vem do nosso DataParser.
        self.data = data
        self.model = gp.Model("Optimal_Wave_Selection")
        
        # Parâmetros de controle da otimização.
        self.lambda_penalty = lambda_penalty
        self.stagnation_node_limit = stagnation_node_limit
        
        # Atributos para armazenar o estado do callback e a solução final.
        self.stagnation_node_counter = 0
        self.last_objective = -float('inf')
        self.solution = {}

    def _stagnation_callback(self, model, where):
        """
        Callback para parar a otimização se não houver melhoria após um certo
        número de nós explorados na árvore de Branch-and-Bound.
        """
        # O callback é acionado a cada nó explorado pelo Gurobi.
        if where == GRB.Callback.MIPNODE:
            # Acessamos estatísticas do nó apenas se uma solução já foi encontrada.
            if model.cbGet(GRB.Callback.MIPNODE_SOLCNT) > 0:
                current_best_obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)
                
                # Se o melhor objetivo melhorou (com uma tolerância), zeramos a contagem.
                if current_best_obj > self.last_objective + 1e-6:
                    self.last_objective = current_best_obj
                    self.stagnation_node_counter = 0
                # Caso contrário, incrementamos.
                else:
                    self.stagnation_node_counter += 1
                
                # Se o limite de estagnação for atingido, terminamos a otimização.
                if self.stagnation_node_counter >= self.stagnation_node_limit:
                    print(f"\nINFO: Limite de estagnação ({self.stagnation_node_limit} nós) atingido. Terminando...")
                    model.terminate()

    def build_and_solve(self):
        """
        Orquestra a construção do modelo, a execução do solver e a extração da solução.
        """
        print("INFO: Construindo o modelo de otimização...")
        
        # --- Passo 1: Criar Variáveis de Decisão ---
        # Para cada pedido/corredor, uma variável binária para decidir se será selecionado.
        self.select_order = self.model.addVars(self.data.all_orders, vtype=GRB.BINARY, name="SelectOrder")
        self.visit_aisle = self.model.addVars(self.data.all_aisles, vtype=GRB.BINARY, name="VisitAisle")

        # --- Passo 2: Definir a Função Objetivo ---
        # Maximizar: (Total de Itens) - lambda * (Total de Corredores)
        total_items_in_wave = gp.quicksum(
            self.data.orders_total_units[o] * self.select_order[o] for o in self.data.all_orders
        )
        total_aisles_visited = gp.quicksum(self.visit_aisle)
        self.model.setObjective(
            total_items_in_wave - self.lambda_penalty * total_aisles_visited, GRB.MAXIMIZE
        )

        # --- Passo 3: Adicionar Restrições ---
        # Restrições de tamanho da wave (LB e UB)
        self.model.addConstr(total_items_in_wave >= self.data.lb, "Min_Wave_Size")
        self.model.addConstr(total_items_in_wave <= self.data.ub, "Max_Wave_Size")

        # Restrições de estoque (usando a estrutura otimizada)
        for item_id in self.data.all_items:
            # Demanda total do item (soma apenas sobre pedidos relevantes)
            total_demand = gp.quicksum(
                quantity * self.select_order[order_id] 
                for order_id, quantity in self.data.item_to_orders.get(item_id, [])
            )
            # Estoque total do item (soma apenas sobre corredores relevantes)
            total_stock = gp.quicksum(
                quantity * self.visit_aisle[aisle_id] 
                for aisle_id, quantity in self.data.item_to_aisles.get(item_id, [])
            )
            # A demanda não pode exceder o estoque.
            self.model.addConstr(total_demand <= total_stock, f"Stock_Constraint_{item_id}")

        # --- Passo 4: Otimizar ---
        print("INFO: Otimização iniciada...")
        self.model.optimize(self._stagnation_callback) # Otimiza usando o callback

        # --- Passo 5: Extrair a Solução ---
        if self.model.SolCount > 0:
            print(f"INFO: Solução final encontrada com valor objetivo = {self.model.ObjVal:.2f}")
            self.solution['selected_orders'] = [o for o, var in self.select_order.items() if var.X > 0.5]
            self.solution['visited_aisles'] = [a for a, var in self.visit_aisle.items() if var.X > 0.5]
        else:
            print("ERRO: Nenhuma solução viável foi encontrada.")

def main(instance_file, output_file):
    """
    Função principal que orquestra todo o processo de ponta a ponta.
    """
    # Parâmetros que podem ser ajustados para experimentar com o modelo.
    lambda_param = 2.0  # Penalidade por corredor.
    stagnation_nodes = 200  # Nº de nós sem melhoria antes de parar.
    time_limit_seconds = 400 # Limite de tempo total em segundos (10 min).

    # --- 2. Parsing ---
    parser = DataParser(instance_file)
    if not parser.parse():
        sys.exit(1) # Termina o script se o parsing falhar.

    # --- 3. Otimização ---
    optimizer = WaveOptimizer(
        parser, 
        lambda_penalty=lambda_param, 
        stagnation_node_limit=stagnation_nodes
    )
    # Define um tempo limite geral para a otimização.
    optimizer.model.setParam('TimeLimit', time_limit_seconds)
    optimizer.build_and_solve()

    # --- 4. Geração da Saída ---
    if optimizer.solution:
        print(f"INFO: Escrevendo a solução para o arquivo: {output_file}")
        try:
            with open(output_file, 'w') as f:
                selected_orders = optimizer.solution['selected_orders']
                visited_aisles = optimizer.solution['visited_aisles']
                
                f.write(f"{len(selected_orders)}\n")
                for order in sorted(selected_orders):
                    f.write(f"{order}\n")
                
                f.write(f"{len(visited_aisles)}\n")
                for aisle in sorted(visited_aisles):
                    f.write(f"{aisle}\n")
            
            print("INFO: Processo concluído com sucesso.")
        except Exception as e:
            print(f"ERRO: Falha ao escrever o arquivo de saída: {e}")
    else:
        print("AVISO: O processo terminou sem uma solução para escrever.")

In [2]:
import os
path_to_instances_b = './datasets/b'
todas_instancias = os.listdir(path_to_instances_b)

In [3]:
from checker_biblioteca.checker import run_checker
'''
for i in range(len(todas_instancias)):
    instance_file = f'{path_to_instances_b}/{todas_instancias[i]}'
    output_file = f'./{todas_instancias[i]}'
    #if __name__ == "__main__":
    main(instance_file, output_file)

    run_checker(instance_file, output_file)
'''

'\nfor i in range(len(todas_instancias)):\n    instance_file = f\'{path_to_instances_b}/{todas_instancias[i]}\'\n    output_file = f\'./{todas_instancias[i]}\'\n    #if __name__ == "__main__":\n    main(instance_file, output_file)\n\n    run_checker(instance_file, output_file)\n'

In [4]:
instance_file = './datasets/b/instance_0011.txt'
output_file = 'solution_0011.txt'
main(instance_file, output_file)
run_checker(instance_file, output_file)

INFO: Iniciando a análise otimizada do arquivo: ./datasets/b/instance_0011.txt
INFO: Análise do arquivo concluída com sucesso.
Set parameter Username
Set parameter LicenseID to value 2638716
Academic license - for non-commercial use only - expires 2026-03-19
Set parameter TimeLimit to value 400
INFO: Construindo o modelo de otimização...


C:\Users\Danone\AppData\Local\Temp\ipykernel_7852\3811473819.py:151: DeprecationWarning: Calling quicksum on a tupledict is deprecated, use .sum() instead.
  total_aisles_visited = gp.quicksum(self.visit_aisle)


INFO: Otimização iniciada...
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  400

Optimize a model with 37822 rows, 45594 columns and 362018 nonzeros
Model fingerprint: 0x0f72d410
Variable types: 0 continuous, 45594 integer (45594 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+03, 2e+04]
Found heuristic solution: objective 18157.000000
Presolve removed 115 rows and 7194 columns
Presolve time: 1.41s
Presolved: 37707 rows, 38400 columns, 301458 nonzeros
Found heuristic solution: objective 18142.000000
Variable types: 0 continuous, 38400 integer (34353 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...


R